<a href="https://colab.research.google.com/github/AdeelAliYousaf/ViolenceDetectionSystem/blob/main/ViolenceDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip uninstall tensorflow keras
!pip install tensorflow==2.15 keras==2.15 skimage gradio

import cv2
import numpy as np
from skimage.transform import resize
from keras.models import load_model
import time
import gradio as gr

model_path = "ViolenceDetectionModel.keras"
model = load_model(model_path, compile=False)
print("[INFO] Pre-trained model loaded successfully.")

def load_video_frames(path, max_frames=30, target_size=(160, 160)):
    cap = cv2.VideoCapture(path)
    frames = []

    while len(frames) < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = resize(frame, target_size, preserve_range=True).astype(np.float32) / 255.0
        frames.append(frame)

    cap.release()

    while len(frames) < max_frames:
        frames.append(np.zeros((target_size[0], target_size[1], 3), dtype=np.float32))

    return np.array(frames)

def predict_violence(video_path, threshold=0.65):
    frames = load_video_frames(video_path)
    frames = np.expand_dims(frames, axis=0)
    start = time.time()
    preds = model.predict(frames)[0]
    end = time.time()

    print(f"[INFO] Prediction took {end - start:.2f}s")
    print(f"[INFO] Predicted probabilities: {preds}")

    is_violent = preds[1] >= threshold
    print(f"[RESULT] Violence detected: {is_violent} (Confidence: {preds[1]:.4f})")

    return is_violent, preds[1]

def gradio_predict(video_file):
    is_fight, confidence = predict_violence(video_file)
    result_text = f"🔴 Violence Detected! (Confidence: {confidence:.2f})" if is_fight else f"🟢 No Violence Detected. (Confidence: {confidence:.2f})"
    return result_text

with gr.Blocks() as demo:
    gr.Markdown("## 🎥 Violence Detection in Video")
    gr.Markdown("Upload a video and click 'Detect Violence'. The model will analyze the clip and display the result.")

    video_input = gr.Video(label="Upload a short video")
    output_text = gr.Textbox(label="Prediction Result", interactive=False)
    predict_button = gr.Button("🚨 Detect Violence")

    predict_button.click(fn=gradio_predict, inputs=video_input, outputs=output_text)

demo.launch(debug=False, share=True)

[INFO] Pre-trained model loaded successfully.
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b07b6d0ce4fe629236.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
